In [1]:
from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH
import mujoco
import pinocchio as pin
import numpy as np

np.random.seed(0)

In [2]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pinmodel.createData()

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

In [3]:
q = np.random.randn(mjmodel.nq)
v = np.random.randn(mjmodel.nv)
dv = np.random.randn(mjmodel.nv)

tau = pin.rnea(pinmodel, pindata, q, v, dv)

mjdata.qpos = q.copy()
mjdata.qvel = v.copy()
# mjdata.qacc = dv.copy()
mjdata.ctrl = tau.copy()

In [4]:
pin.computeAllTerms(pinmodel, pindata, q, v)
pin.forwardKinematics(pinmodel, pindata, q, v, dv)
pin.framesForwardKinematics(pinmodel, pindata, q)

mujoco.mj_step(mjmodel, mjdata)

In [5]:
def pin_body(idx):
    vel = pin.getVelocity(pinmodel, pindata, idx, pin.LOCAL)

    accel = pin.getAcceleration(pinmodel, pindata, idx, pin.LOCAL)

    rotation = pindata.oMi[idx].rotation

    return vel, accel, rotation


body_idx = 4

vel, acc, rotation = pin_body(body_idx)

vel, acc, rotation

(  v = -0.0386146  -0.213597 0.00697333
   w =   0.43721  0.156022 -0.843514,
   v = 0.0825157 -0.141572 0.0439487
   w =   0.48182  0.899213 -0.607455,
 array([[ 0.17051182, -0.9813841 ,  0.08837964],
        [-0.87129896, -0.19205531, -0.45161143],
        [ 0.46017805,  0.        , -0.88782665]]))

In [6]:
# velocity computation 2 - xbody
rotlin_vel = np.zeros(6)
mujoco.mj_objectVelocity(mjmodel, mjdata, 2, body_idx + 1, rotlin_vel, 1)

# acceleration computation
mujoco.mj_rnePostConstraint(mjmodel, mjdata)
rotlin_acc = np.zeros(6)
mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, body_idx + 1, rotlin_acc, 1)

# rotation matrix
rotation = mjdata.xmat[body_idx + 1].reshape(3, 3).copy()
rotlin_vel, rotlin_acc, rotation

(array([ 0.43720984,  0.15602244, -0.84351382, -0.03861464, -0.21359706,
         0.00697333]),
 array([-4.05971161e+01, -2.64432206e+04,  7.80043987e+01, -3.28416499e+02,
         1.98553895e+01, -9.16986970e+02]),
 array([[ 1.70511825e-01, -9.81384103e-01,  8.83796400e-02],
        [-8.71298959e-01, -1.92055313e-01, -4.51611426e-01],
        [ 4.60178054e-01,  8.32667268e-17, -8.87826649e-01]]))